# Ticket Sharpening (Work Notion) via ultimate-notion (UNO)

Status: WIP
Owner: Hugo Evers
GitHub issue: https://github.com/hugocool/FateForger/issues/16
Issue branch: issue/14-tasks-agent-ticktick-lists
PR: TBD
notebook-mode: notebook-mode
Last clean run: 2026-02-17 (pending) (.venv, Python 3.11.9)
Repo cleanliness snapshot: dirty (see `git status --porcelain`)

Targets (work Notion)
- Tasks DBs: `110baca1-857f-48b1-a8ec-cea325202eef` (big), `d0a4103c-d10a-4860-9a71-6bf4fc55192f` (small)
- Projects DBs: `c37cbc40-020e-40db-88eb-d460e266b08b` (big), `9f34f42d-d3d4-449d-8d39-ee7d529fbfd4` (small)

Notes
- Execution status stays in existing task `Status` (Not started / In progress / Done / Archived).
- Ticket refinement/triage is handled via a separate `Ticket Status` property (added if missing).


## Pairing Intake Record

Problem
- Some tasks enable fake deep work: ambiguous titles and missing binary DoD.
- We want to surgically sharpen only a small set of priority tickets + stale C2F automation guilt threads.
- Projects should cover the full breadth (Finances, Marketing, Sales, BizDev) so tasks can be routed and Work Type derived.

Constraints
- Work tenant only, via `WORK_NOTION_TOKEN` from `.env`.
- Non-destructive: no deletes; no renaming existing properties; do not rewrite existing task `Status` options.
- Default dry-run; only mutate Notion when `APPLY_CHANGES=True`.
- DoD may live in page body and/or in properties; this notebook prefers properties when available.


## Design Options

Option A (recommended): Property-only sharpening
- Update: Task title, DoD property (if present), Summary (scope cap / artifact pointer), Ticket Status.
- Create: missing Projects, and missing priority Tasks if they do not exist.
- Avoid: editing page body blocks.

Option B: Page-body editing
- Update page children to enforce a DoD template.
- Higher risk; harder to keep surgical.

Selected
- Option A.


## Implementation Walkthrough / Decision Audit

Priority tickets to sharpen
- Outbound/pipeline: Bart outreach (single artifact: outbound sent).
- Gerimedica: this week is integration plan + alignment, not full deploy.
- C2F automation: collapse stale threads into one v0 experiment ticket; pause the rest.
- Events: commit to 1 event this month + 1 next month.

Project breadth
- Ensure Projects DBs include: Finances, Marketing, Sales, BizDev.
- Add `Work Type` (select) on Projects; tasks derive `Work Type` via Project rollup when possible.

Non-goals
- TickTick sync later.
- Notion view creation is manual.


In [ ]:
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, Optional

import ultimate_notion as uno
from ultimate_notion import Session, schema

from fateforger.setup_wizard.envfile import read_env_file


# Safety gate: keep False until you review the dry-run plan.
APPLY_CHANGES = False

ENV = read_env_file(Path(".env"))
WORK_TOKEN = (ENV.get("WORK_NOTION_TOKEN") or "").strip()
if not WORK_TOKEN:
    raise RuntimeError("WORK_NOTION_TOKEN missing in .env")

# ultimate-notion reads token from its config/env; it ignores Session(auth=...).
# Hack: set env var before creating the Session.
os.environ["NOTION_TOKEN"] = WORK_TOKEN
Session._active_session = None  # type: ignore[attr-defined]

TASKS_BIG = "110baca1-857f-48b1-a8ec-cea325202eef"
TASKS_SMALL = "d0a4103c-d10a-4860-9a71-6bf4fc55192f"
PROJECTS_BIG = "c37cbc40-020e-40db-88eb-d460e266b08b"
PROJECTS_SMALL = "9f34f42d-d3d4-449d-8d39-ee7d529fbfd4"

WORK_TYPE_OPTIONS = ["Revenue", "Product", "Research", "System", "Visibility"]
TICKET_STATUS_OPTIONS = ["Unrefined", "Refined", "Ready", "Paused", "Zombie", "Blocked"]

NEW_PROJECT_PAGES = ["Finances", "Marketing", "Sales", "BizDev"]


@dataclass(frozen=True)
class PlannedChange:
    db_id: str
    page_id: Optional[str]
    action: str
    details: str


def _print_plan(planned: list[PlannedChange]) -> None:
    if not planned:
        print("No changes needed.")
        return
    print("Planned changes:")
    for ch in planned:
        suffix = f" page={ch.page_id}" if ch.page_id else ""
        print(f"- [{ch.db_id}]{suffix} {ch.action}: {ch.details}")


def _norm(s: str) -> str:
    return (s or "").strip().lower()


In [ ]:
notion = Session.get_or_create()

tasks_big = notion.get_db(TASKS_BIG, use_cache=False)
tasks_small = notion.get_db(TASKS_SMALL, use_cache=False)
projects_big = notion.get_db(PROJECTS_BIG, use_cache=False)
projects_small = notion.get_db(PROJECTS_SMALL, use_cache=False)

print("Loaded:")
print("-", tasks_big.title, tasks_big.id)
print("-", tasks_small.title, tasks_small.id)
print("-", projects_big.title, projects_big.id)
print("-", projects_small.title, projects_small.id)

print("\nTasks (big) schema:\n", tasks_big.schema)
print("\nTasks (small) schema:\n", tasks_small.schema)
print("\nProjects (big) schema:\n", projects_big.schema)
print("\nProjects (small) schema:\n", projects_small.schema)


In [ ]:
def ensure_select_property(
    db: uno.database.Database,
    *,
    prop_name: str,
    options: Iterable[str],
    planned: list[PlannedChange],
) -> None:
    if db.schema.has_prop(prop_name):
        return
    planned.append(
        PlannedChange(
            db_id=str(db.id),
            page_id=None,
            action="add_property",
            details=f"{prop_name} (select) options={list(options)}",
        )
    )
    if not APPLY_CHANGES:
        return
    db.schema[prop_name] = schema.Select(options=[schema.Option(o) for o in options])


def ensure_rollup_work_type(
    *,
    tasks_db: uno.database.Database,
    projects_db: uno.database.Database,
    planned: list[PlannedChange],
) -> None:
    rollup_name = "Work Type"
    if tasks_db.schema.has_prop(rollup_name):
        return
    if not tasks_db.schema.has_prop("Project"):
        planned.append(
            PlannedChange(
                db_id=str(tasks_db.id),
                page_id=None,
                action="skip_rollup",
                details="No Project relation found",
            )
        )
        return
    if not projects_db.schema.has_prop("Work Type"):
        planned.append(
            PlannedChange(
                db_id=str(tasks_db.id),
                page_id=None,
                action="skip_rollup",
                details="Projects DB missing Work Type; add it first",
            )
        )
        return
    planned.append(
        PlannedChange(
            db_id=str(tasks_db.id),
            page_id=None,
            action="add_property",
            details="Work Type (rollup: Project -> Work Type)",
        )
    )
    if not APPLY_CHANGES:
        return
    tasks_db.schema[rollup_name] = schema.Rollup(
        relation=tasks_db.schema.get_prop("Project"),
        rollup=projects_db.schema.get_prop("Work Type"),
    )


def ensure_project_pages(
    projects_db: uno.database.Database,
    *,
    names: Iterable[str],
    planned: list[PlannedChange],
) -> None:
    def _canon(value: str) -> str:
        return "".join(ch for ch in (value or "").lower() if ch.isalnum())

    existing = {_canon(p.title) for p in projects_db.get_all_pages()}
    title_attr = projects_db.schema.get_title_prop().attr_name
    for name in names:
        if _canon(name) in existing:
            continue
        planned.append(
            PlannedChange(
                db_id=str(projects_db.id),
                page_id=None,
                action="create_project",
                details=name,
            )
        )
        if APPLY_CHANGES:
            projects_db.create_page(**{title_attr: name})
        existing.add(_canon(name))


planned: list[PlannedChange] = []

# Projects: Work Type + breadth pages.
for pdb in (projects_big, projects_small):
    ensure_select_property(pdb, prop_name="Work Type", options=WORK_TYPE_OPTIONS, planned=planned)
    ensure_project_pages(pdb, names=NEW_PROJECT_PAGES, planned=planned)

# Tasks: Ticket Status + Work Type rollup.
for tdb, pdb in ((tasks_big, projects_big), (tasks_small, projects_small)):
    ensure_select_property(tdb, prop_name="Ticket Status", options=TICKET_STATUS_OPTIONS, planned=planned)
    ensure_rollup_work_type(tasks_db=tdb, projects_db=pdb, planned=planned)

_print_plan(planned)
print("\nAPPLY_CHANGES=", APPLY_CHANGES)


In [ ]:
def _proj_name(task_page: uno.page.Page) -> str:
    proj = task_page.props.get("Project")
    if isinstance(proj, list) and proj:
        return proj[0].title
    return ""


def _assignees(task_page: uno.page.Page) -> str:
    ppl = task_page.props.get("Assignee")
    if isinstance(ppl, list) and ppl:
        return ", ".join(getattr(p, "name", "") or p.id for p in ppl)
    return ""


def _status(task_page: uno.page.Page) -> str:
    return str(task_page.props.get("Status") or "")


def _ticket_status(task_page: uno.page.Page) -> str:
    v = task_page.props.get("Ticket Status")
    return str(v or "")


def _dod_or_summary(task_page: uno.page.Page, *, has_dod: bool) -> str:
    if has_dod:
        return str(task_page.props.get("DoD") or "")
    return str(task_page.props.get("Summary") or "")


def dump_in_progress(db: uno.database.Database) -> list[uno.page.Page]:
    pages = db.get_all_pages()
    inprog = [p for p in pages if _norm(_status(p)) == "in progress"]
    print(f"\nIn progress ({db.id}) count={len(inprog)}")
    has_dod = db.schema.has_prop("DoD")
    for p in inprog:
        dod = _dod_or_summary(p, has_dod=has_dod)
        dod_short = (dod[:120] + "...") if len(dod) > 120 else dod
        print(f"- {p.title} | project={_proj_name(p)} | owner={_assignees(p)} | ticket_status={_ticket_status(p)} | dod/summary={dod_short}")
    return inprog


inprog_big = dump_in_progress(tasks_big)
inprog_small = dump_in_progress(tasks_small)


In [ ]:
@dataclass(frozen=True)
class RewriteSpec:
    key: str
    match_substrings: list[str]
    new_title: str
    dod: str
    scope_cap: str
    ticket_status: str
    project_hint: str
    require_in_progress: bool = True


@dataclass(frozen=True)
class CreateSpec:
    key: str
    new_title: str
    dod: str
    scope_cap: str
    ticket_status: str
    project_hint: str


@dataclass(frozen=True)
class PauseSpec:
    key: str
    match_substrings: list[str]
    ticket_status: str
    pause_note: str
    set_execution_status: str = "Not started"


# Only rewrite tasks that are currently In progress (to stay surgical).
REWRITE_SPECS = [
    RewriteSpec(
        key="outbound_bart",
        match_substrings=["bart geerts benaderen"],
        new_title="Send Bart Geerts pain-point message (outbound artifact)",
        dod="Message sent to Bart Geerts (and follow-up reminder created).",
        scope_cap="One message. Max 15 minutes drafting.",
        ticket_status="Ready",
        project_hint="commercie",
    ),
    RewriteSpec(
        key="gerimedica_plan",
        match_substrings=["c2f demo deployen"],
        new_title="Gerimedica integration plan v1 (2 pages) + alignment",
        dod="2-page plan exists + alignment meeting scheduled/held + owners/blockers captured.",
        scope_cap="Doc max 2 pages. No deployment work in this ticket.",
        ticket_status="Refined",
        project_hint="gerimedica",
    ),
]


# Create v0/commitment tickets even if older vague threads exist.
CREATE_SPECS = [
    CreateSpec(
        key="kg_loop_v0",
        new_title="KG extraction loop v0 on one example set",
        dod="1 loop executed + outputs saved + 1/2-page note written.",
        scope_cap="One dataset. One loop. No generalization.",
        ticket_status="Refined",
        project_hint="corpus",
    ),
    CreateSpec(
        key="events_commit",
        new_title="Commit to 1 event this month + 1 next month",
        dod="Registered/RSVPd for 1 event + selected next month event + 10-line process note.",
        scope_cap="Max 15 minutes research per event.",
        ticket_status="Ready",
        project_hint="biz dev",
    ),
]


# Stale automation threads: mark as Zombie and bump out of execution WIP.
PAUSE_SPECS = [
    PauseSpec(
        key="stale_taxonomy_threads",
        match_substrings=[
            "automating the taxonomy generation",
            "automated facet uit elkaar halen",
            "taxonomy extraction prompt modifications",
            "notebook.widget to help construct",
        ],
        ticket_status="Zombie",
        pause_note="Collapsed into KG extraction loop v0 ticket; resume only if v0 proves value.",
    ),
    PauseSpec(
        key="systems_pause_until_friday",
        match_substrings=[
            "weekly review",
            "timeboxing agent",
        ],
        ticket_status="Paused",
        pause_note="Paused until 2026-02-20 (Friday). Resume trigger: after outbound + Gerimedica plan shipped.",
    ),
]


def _page_status(p: uno.page.Page) -> str:
    return str(p.props.get("Status") or "")


def _matches_all(title: str, subs: list[str]) -> bool:
    t = _norm(title)
    return all(_norm(s) in t for s in subs)


def find_best_match(db: uno.database.Database, spec: RewriteSpec) -> Optional[uno.page.Page]:
    best: Optional[uno.page.Page] = None
    best_score = -1
    subs = [_norm(s) for s in spec.match_substrings]
    for p in db.get_all_pages():
        title = p.title or ""
        t = _norm(title)
        hit_count = sum(1 for s in subs if s and s in t)
        if hit_count == 0:
            continue
        status = _norm(_page_status(p))
        if spec.require_in_progress and status != "in progress":
            continue
        score = hit_count * 10
        if _matches_all(title, subs):
            score += 1000
        if status == "in progress":
            score += 100
        if score > best_score:
            best = p
            best_score = score
    return best


def find_first_by_title(db: uno.database.Database, title: str) -> Optional[uno.page.Page]:
    want = _norm(title)
    for p in db.get_all_pages():
        if _norm(p.title) == want:
            return p
    return None


def find_in_progress_matches(db: uno.database.Database, substrings: list[str]) -> list[uno.page.Page]:
    subs = [_norm(s) for s in substrings]
    out: list[uno.page.Page] = []
    for p in db.get_all_pages():
        if _norm(_page_status(p)) != "in progress":
            continue
        t = _norm(p.title)
        if any(s in t for s in subs if s):
            out.append(p)
    return out


def build_rewrite_plan() -> list[PlannedChange]:
    plan: list[PlannedChange] = []
    for spec in REWRITE_SPECS:
        match = find_best_match(tasks_big, spec) or find_best_match(tasks_small, spec)
        if match:
            plan.append(
                PlannedChange(
                    db_id=str(match.parent_db.id),
                    page_id=str(match.id),
                    action="update_task",
                    details=f"{match.title!r} -> {spec.new_title!r}; ticket_status={spec.ticket_status}; DoD/Summary updated",
                )
            )

    for spec in CREATE_SPECS:
        existing = find_first_by_title(tasks_big, spec.new_title) or find_first_by_title(tasks_small, spec.new_title)
        if existing:
            plan.append(
                PlannedChange(
                    db_id=str(existing.parent_db.id),
                    page_id=str(existing.id),
                    action="keep_existing",
                    details=f"{spec.new_title!r} already exists",
                )
            )
        else:
            plan.append(
                PlannedChange(
                    db_id=str(tasks_big.id),
                    page_id=None,
                    action="create_task",
                    details=f"{spec.new_title!r}; ticket_status={spec.ticket_status}; DoD/Summary set",
                )
            )

    for spec in PAUSE_SPECS:
        matches = find_in_progress_matches(tasks_big, spec.match_substrings)
        for p in matches:
            plan.append(
                PlannedChange(
                    db_id=str(tasks_big.id),
                    page_id=str(p.id),
                    action="pause_thread",
                    details=f"{p.title!r} -> ticket_status={spec.ticket_status}; status={spec.set_execution_status}",
                )
            )
    return plan


rewrite_plan = build_rewrite_plan()
_print_plan(rewrite_plan)


In [ ]:
def _set_task_props(
    *,
    task: uno.page.Page,
    new_title: str,
    ticket_status: str,
    dod: str,
    scope_cap: str,
) -> None:
    db = task.parent_db
    assert db is not None

    title_attr = db.schema.get_title_prop().attr_name
    payload = {
        title_attr: new_title,
    }

    # Ticket Status may not exist yet (if you didn't apply schema changes).
    if db.schema.has_prop("Ticket Status"):
        payload[db.schema.get_prop("Ticket Status").attr_name] = ticket_status

    if db.schema.has_prop("DoD"):
        payload[db.schema.get_prop("DoD").attr_name] = dod
        # Keep Summary as scope cap / artifact pointer.
        if db.schema.has_prop("Summary"):
            payload[db.schema.get_prop("Summary").attr_name] = f"Scope cap: {scope_cap}"
    else:
        # Small board has no DoD property; store both in Summary.
        if db.schema.has_prop("Summary"):
            payload[db.schema.get_prop("Summary").attr_name] = f"DoD: {dod}\nScope cap: {scope_cap}"

    task.update_props(**payload)


def apply_specs() -> None:
    if not APPLY_CHANGES:
        print("APPLY_CHANGES is False; skipping writes.")
        return

    def find_project(projects_db: uno.database.Database, hint: str) -> Optional[uno.page.Page]:
        h = _norm(hint)
        if not h:
            return None
        for p in projects_db.get_all_pages():
            if h in _norm(p.title):
                return p
        return None

    def ensure_project_on_task(task: uno.page.Page, proj: Optional[uno.page.Page]) -> None:
        if not proj:
            return
        db = task.parent_db
        assert db is not None
        current = task.props.get("Project")
        if isinstance(current, list) and current:
            return
        if not db.schema.has_prop("Project"):
            return
        task.update_props(**{db.schema.get_prop("Project").attr_name: [proj]})

    # 1) Rewrites (must already be In progress).
    for spec in REWRITE_SPECS:
        match = find_best_match(tasks_big, spec) or find_best_match(tasks_small, spec)
        if not match:
            continue
        _set_task_props(
            task=match,
            new_title=spec.new_title,
            ticket_status=spec.ticket_status,
            dod=spec.dod,
            scope_cap=spec.scope_cap,
        )
        # Only set Project when empty (avoid clobbering).
        if match.parent_db.id == tasks_big.id:
            ensure_project_on_task(match, find_project(projects_big, spec.project_hint))
        else:
            ensure_project_on_task(match, find_project(projects_small, spec.project_hint))

    # 2) Creates (v0/commitment tickets).
    created_or_existing: dict[str, uno.page.Page] = {}
    for spec in CREATE_SPECS:
        existing = find_first_by_title(tasks_big, spec.new_title) or find_first_by_title(tasks_small, spec.new_title)
        if existing:
            created_or_existing[spec.key] = existing
            continue

        title_attr = tasks_big.schema.get_title_prop().attr_name
        kwargs = {
            title_attr: spec.new_title,
            tasks_big.schema.get_prop("Status").attr_name: "Not started",
        }
        if tasks_big.schema.has_prop("Ticket Status"):
            kwargs[tasks_big.schema.get_prop("Ticket Status").attr_name] = spec.ticket_status
        if tasks_big.schema.has_prop("DoD"):
            kwargs[tasks_big.schema.get_prop("DoD").attr_name] = spec.dod
        if tasks_big.schema.has_prop("Summary"):
            kwargs[tasks_big.schema.get_prop("Summary").attr_name] = f"Scope cap: {spec.scope_cap}"
        proj = find_project(projects_big, spec.project_hint)
        if proj and tasks_big.schema.has_prop("Project"):
            kwargs[tasks_big.schema.get_prop("Project").attr_name] = [proj]

        created = tasks_big.create_page(**kwargs)
        created_or_existing[spec.key] = created

    # 3) Pause/Zombie threads (and bump them out of execution WIP).
    kg_parent = created_or_existing.get("kg_loop_v0")
    for spec in PAUSE_SPECS:
        for p in find_in_progress_matches(tasks_big, spec.match_substrings):
            db = p.parent_db
            assert db is not None

            updates = {}
            if db.schema.has_prop("Ticket Status"):
                updates[db.schema.get_prop("Ticket Status").attr_name] = spec.ticket_status
            if db.schema.has_prop("Status"):
                updates[db.schema.get_prop("Status").attr_name] = spec.set_execution_status
            if db.schema.has_prop("Summary"):
                link = f"\nParent: {kg_parent.url}" if (kg_parent and spec.ticket_status == "Zombie") else ""
                updates[db.schema.get_prop("Summary").attr_name] = f"{spec.pause_note}{link}"

            if updates:
                p.update_props(**updates)

            # If this is a zombie taxonomy thread, link it under KG v0 via Parent-task when possible.
            if kg_parent and spec.ticket_status == "Zombie" and db.schema.has_prop("Parent-task"):
                p.update_props(**{db.schema.get_prop("Parent-task").attr_name: [kg_parent]})

    print("Applied ticket sharpening updates.")


apply_specs()


## Reviewer Checklist

- Run from top with `APPLY_CHANGES=False` and review:
  - In-progress list
  - Planned changes list
- If correct, set `APPLY_CHANGES=True` and rerun:
  - Schema adds (Work Type, Ticket Status, rollup)
  - Project pages created if missing
  - Only a small set of tasks updated/created

Manual steps (Notion UI)
- Create view: In Progress + Ticket Status in (Unrefined, Zombie)
- Create view: Revenue lane (Work Type=Revenue AND Status in (Not started, In progress))


## Open Items

To decide
- Exact canonical naming: "Bart Geerts" vs other variant.
- Whether to backfill Work Type on existing projects now.

To do
- If `systems_pause` matches a real ticket, verify the pause date is correct.

Blocked by
- none


## Acceptance Criteria Checklist

- [ ] Notebook scaffold exists and maps to GitHub issue.
- [ ] Dry-run produces plan and makes no writes.
- [ ] Apply mode updates only a small, explicit set of tasks and adds only new properties/pages.
- [ ] Projects include Finances/Marketing/Sales/BizDev.


## Implementation Evidence

- Evidence is the notebook output (planned change list) and the Notion UI after apply.


## Extraction Map (Notebook -> Artifacts)

- Notebook-only by design; this is operational runbook work.


## Closeout / Remaining Notebook-Only Content

- Keep as a runbook until multi-tenant Notion support is implemented in production code.
